# Google Drive

>[Google Drive](https://en.wikipedia.org/wiki/Google_Drive) 是由 Google 开发的文件存储和同步服务。

本 Notebook 涵盖了如何从 `Google Drive` 加载文档。当前仅支持 `Google Docs`。

## 前提条件

1. 创建一个 Google Cloud 项目或使用现有项目
1. 启用 [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)
1. [授权桌面应用凭据](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)
1. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`

## 🧑 提取 Google Docs 数据的说明
将环境变量 `GOOGLE_APPLICATION_CREDENTIALS` 设置为空字符串 (`""`)。

默认情况下，`GoogleDriveLoader` 期望 `credentials.json` 文件位于 `~/.credentials/credentials.json`，但这可以通过 `credentials_path` 关键字参数进行配置。`token.json` 也是一样——默认路径：`~/.credentials/token.json`，构造函数参数：`token_path`。

首次使用 `GoogleDriveLoader` 时，浏览器中会显示同意屏幕以进行用户身份验证。身份验证后，`token.json` 将在提供的或默认路径下自动创建。另外，如果该路径下已存在 `token.json`，则不会提示进行身份验证。

`GoogleDriveLoader` 可以从 Google Docs 文档 ID 列表或文件夹 ID 加载。您可以从 URL 获取文件夹和文档 ID：

* Folder: https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> folder id is `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"`
* Document: https://docs.google.com/document/d/1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> document id is `"1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw"`

In [ ]:
%pip install --upgrade --quiet langchain-google-community[drive]

In [ ]:
from langchain_google_community import GoogleDriveLoader

In [ ]:
loader = GoogleDriveLoader(
    folder_id="1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5",
    token_path="/path/where/you/want/token/to/be/created/google_token.json",
    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False,
)

In [ ]:
docs = loader.load()

当你通过 `folder_id` 时，默认会加载所有类型为 document、sheet 和 pdf 的文件。你可以通过传递 `file_types` 参数来修改此行为。

In [ ]:
loader = GoogleDriveLoader(
    folder_id="1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5",
    file_types=["document", "sheet"],
    recursive=False,
)

## 传入可选的文件加载器

在处理 Google Docs 和 Google Sheets 以外的文件时，将可选的文件加载器传递给 `GoogleDriveLoader` 会很有帮助。如果传入了文件加载器，该文件加载器将用于没有 Google Docs 或 Google Sheets MIME 类型的文档。下面是如何使用文件加载器从 Google Drive 加载 Excel 文档的示例。

In [ ]:
from langchain_community.document_loaders import UnstructuredFileIOLoader
from langchain_google_community import GoogleDriveLoader

In [ ]:
file_id = "1x9WBtFPWMEAdjcJzPScRsjpjQvpSo_kz"
loader = GoogleDriveLoader(
    file_ids=[file_id],
    file_loader_cls=UnstructuredFileIOLoader,
    file_loader_kwargs={"mode": "elements"},
)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

您也可以使用下面的模式来处理混合了文件和 Google Docs/Sheets 的文件夹：

In [ ]:
folder_id = "1asMOHY1BqBS84JcRbOag5LOJac74gpmD"
loader = GoogleDriveLoader(
    folder_id=folder_id,
    file_loader_cls=UnstructuredFileIOLoader,
    file_loader_kwargs={"mode": "elements"},
)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

## 扩展用法
一个外部（非官方）组件可以管理 Google Drive 的复杂性：`langchain-googledrive`
它与 `langchain_community.document_loaders.GoogleDriveLoader` 兼容，可以
替代它使用。

为了兼容容器，认证使用环境变量 `GOOGLE_ACCOUNT_FILE` 来存储凭证文件（用户或服务）。

In [ ]:
%pip install --upgrade --quiet  langchain-googledrive

In [ ]:
folder_id = "root"
# folder_id='1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5'

In [ ]:
# Use the advanced version.
from langchain_googledrive.document_loaders import GoogleDriveLoader

In [ ]:
loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    num_results=2,  # Maximum number of file to load
)

默认情况下，所有具有这些mime 类型的文件都可以转换为 `Document`。
- text/text
- text/plain
- text/html
- text/csv
- text/markdown
- image/png
- image/jpeg
- application/epub+zip
- application/pdf
- application/rtf
- application/vnd.google-apps.document (GDoc)
- application/vnd.google-apps.presentation (GSlide)
- application/vnd.google-apps.spreadsheet (GSheet)
- application/vnd.google.colaboratory (Notebook colab)
- application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)
- application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)

可以更新或自定义此设置。请参阅 `GDriveLoader` 的文档。

但必须安装相应的包。

In [ ]:
%pip install --upgrade --quiet  unstructured

In [ ]:
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

### 加载授权身份

Google Drive Loader 摄取的每个文件的授权身份都可以与每个 Document 的元数据一起加载。

In [ ]:
from langchain_google_community import GoogleDriveLoader

loader = GoogleDriveLoader(
    folder_id=folder_id,
    load_auth=True,
    # Optional: configure whether to load authorized identities for each Document.
)

doc = loader.load()

您可以传递 `load_auth=True`，将 Google Drive 文档访问身份添加到元数据中。

In [ ]:
doc[0].metadata

### 加载扩展元数据
以下额外字段也可以在每个 Document 的元数据中获取：
- full_path - 文件在 Google Drive 中的完整路径。
- owner - 文件所有者。
- size - 文件大小。

In [ ]:
from langchain_google_community import GoogleDriveLoader

loader = GoogleDriveLoader(
    folder_id=folder_id,
    load_extended_matadata=True,
    # Optional: configure whether to load extended metadata for each Document.
)

doc = loader.load()

您可以传递 `load_extended_matadata=True` 来为元数据添加 Google Drive 文档的扩展详细信息。

In [ ]:
doc[0].metadata

### 自定义搜索模式

所有与 Google [`list()`](https://developers.google.com/drive/api/v3/reference/files/list) API 兼容的参数都可以设置。

要指定 Google 请求的新模式，您可以使用 `PromptTemplate()`。
提示的变量可以在构造函数中使用 `kwargs` 设置。
提供了一些预格式化的请求（使用 `{query}`、`{folder_id}` 和/或 `{mime_type}`）：

您可以自定义用于选择文件的条件。提供了一组预定义的过滤器：

| template                               | description                                                           |
| -------------------------------------- | --------------------------------------------------------------------- |
| gdrive-all-in-folder                   | 从 `folder_id` 返回所有兼容文件                                        |
| gdrive-query                           | 在所有驱动器中搜索 `query`                                            |
| gdrive-by-name                         | 按名称搜索文件 `query`                                                |
| gdrive-query-in-folder                 | 在 `folder_id` 中搜索 `query`（如果 `recursive=true`，则包括子文件夹） |
| gdrive-mime-type                       | 搜索特定的 `mime_type`                                                |
| gdrive-mime-type-in-folder             | 在 `folder_id` 中搜索特定的 `mime_type`                               |
| gdrive-query-with-mime-type            | 使用特定的 `mime_type` 搜索 `query`                                   |
| gdrive-query-with-mime-type-and-folder | 使用特定的 `mime_type` 在 `folder_id` 中搜索 `query`                  |

In [ ]:
loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    template="gdrive-query",  # Default template to use
    query="machine learning",
    num_results=2,  # Maximum number of file to load
    supportsAllDrives=False,  # GDrive `list()` parameter
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

您可以自定义您的模式。

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate

loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    template=PromptTemplate(
        input_variables=["query", "query_name"],
        template="fullText contains '{query}' and name contains '{query_name}' and trashed=false",
    ),  # Default template to use
    query="machine learning",
    query_name="ML",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

转换可以在 Markdown 格式中管理：
- 项目符号
- 链接
- 表格
- 标题

将属性 `return_link` 设置为 `True` 以导出链接。

#### GSlide 和 GSheet 的模式
`mode` 参数接受不同的值：

- "document"：返回每个文档的正文
- "snippets"：返回每个文件的描述（在 Google Drive 文件元数据中设置）。


`gslide_mode` 参数接受不同的值：

- "single"：一个文档，包含 `<PAGE BREAK>`
- "slide"：每个幻灯片一个文档
- "elements"：每个元素一个文档。

In [ ]:
loader = GoogleDriveLoader(
    template="gdrive-mime-type",
    mime_type="application/vnd.google-apps.presentation",  # Only GSlide files
    gslide_mode="slide",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

`gsheet_mode` 参数接受不同的值：

- `"single"`：为每一行生成一个文档
- `"elements"`：生成一个包含 markdown 数组以及 `<PAGE BREAK>` 标签的文档。

In [ ]:
loader = GoogleDriveLoader(
    template="gdrive-mime-type",
    mime_type="application/vnd.google-apps.spreadsheet",  # Only GSheet files
    gsheet_mode="elements",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

### 高级用法
所有 Google 文件在元数据中都有一个 `description` 字段。此字段可用于存储文档摘要或其他索引标签（请参阅 `lazy_update_description_with_summary()` 方法）。

如果使用 `mode="snippet"`，则仅使用 description 作为正文。否则，将使用 `metadata['summary']` 字段。

有时，可以使用特定的过滤器从文件名中提取信息，以根据特定标准选择某些文件。您可以使用过滤器。

有时会返回许多文档。不必同时将所有文档保留在内存中。您可以使用方法的惰性版本，一次获取一个文档。最好使用复杂的查询而不是递归搜索。对于每个文件夹，如果启用了 `recursive=True`，则必须应用查询。

In [ ]:
import os

loader = GoogleDriveLoader(
    gdrive_api_file=os.environ["GOOGLE_ACCOUNT_FILE"],
    num_results=2,
    template="gdrive-query",
    filter=lambda search, file: "#test" not in file.get("description", ""),
    query="machine learning",
    supportsAllDrives=False,
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")